In [ ]:
%load 1.1.train_model.py
%run 1.1.train_model.py

/root/miniconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
validation loss = 1.489493: 100%|██████████| 47/47 [00:03<00:00, 15.19it/s]



epoch 1, best valid loss = inf,count = 0


validation loss = 1.400514: 100%|██████████| 47/47 [00:03<00:00, 15.41it/s]



epoch 2, best valid loss = inf,count = 0


validation loss = 1.378233: 100%|██████████| 47/47 [00:03<00:00, 15.01it/s]


...

validation loss = 1.283520: 100%|██████████| 47/47 [00:03<00:00, 15.29it/s]



epoch 90, best valid loss = 59.58388066,count = 4


validation loss = 1.273405: 100%|██████████| 47/47 [00:03<00:00, 15.47it/s]



epoch 91, best valid loss = 59.58388066,count = 5


validation loss = 1.277613:  34%|███▍      | 16/47 [00:01<00:01, 17.05it/s]

In [ ]:
!/usr/bin/shutdown